In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("project1").getOrCreate()
spark.sparkContext.getConf().getAll()

In [ ]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").option("nullvalue","NA").option("escape","\"").option("path","fake_job_postings.csv").load()

In [ ]:
df=df.dropna(subset='salary_range')

In [ ]:
from pyspark.sql.functions import udf
mini=udf(lambda x : x.partition('-')[0])
maxi=udf(lambda x : x.partition('-')[2])

In [ ]:
df = df.withColumn('min',mini(df.salary_range))
df = df.withColumn('max',maxi(df.salary_range))

In [ ]:
df.printSchema

In [ ]:
#converting min and max columns to integer types
from pyspark.sql.types import IntegerType
df = df.withColumn('min',df["min"].cast('int'))
df = df.withColumn('max',df["max"].cast('int'))

In [ ]:
df.printSchema

In [ ]:
df_fake=df.select("salary_range","min","max").filter("fraudulent==1")
df_real=df.select("salary_range","min","max").filter("fraudulent==0")

In [ ]:
approx_median = df_fake.approxQuantile('min',[0.5],0)

In [ ]:
approx_median

In [ ]:
#question 1 : computing the mean and standard deviation of fake job postings
df_fake.describe('max').show()

In [ ]:
#question2
approxmedian_rj = df_real.approxQuantile('min',[0.5],0)

In [ ]:
approxmedian_rj

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
w = Window.orderBy('min')
df_real = df_real.withColumn('id',row_number().over(w))

In [ ]:
df_real.show(20)

In [ ]:
#mid_id
round(df_real.count()/2) + 1

In [ ]:
#i have identified the outliers to be seven figure salaries, dates and three figure salaries
df_real.select('min').filter('id==1323').show()

In [ ]:
#removing outliers
#removing dates
df=df.dropna(subset=['min','max'])

In [ ]:
df1 = df.filter('min>1000'and 'max>1000')
df1 = df1.filter('min<500000'and 'max<500000')

In [ ]:
df_fake1=df1.select("salary_range","min","max").filter("fraudulent==1")
df_real1=df1.select("salary_range","min","max").filter("fraudulent==0")

In [ ]:
df_real1.show()

In [ ]:
df_fake1.describe('max').show()

In [ ]:
#question2
approxmedian_rjn = df_real1.approxQuantile('min',[0.5],0)

In [ ]:
approxmedian_rjn

In [ ]:
df_real1.count()/2

In [ ]:
df_real1 = df_real1.withColumn('id',row_number().over(w))
df_real1.select('min').filter('id==1119').show()

In [ ]:
dfn=df = spark.read.format("csv").option("header","true").option("inferSchema","true").option("nullvalue","NA").option("escape","\"").option("path","fake_job_postings.csv").load()

In [ ]:
rdd_real=dfn.select('description','requirements').filter('fraudulent==0').fillna(' ').rdd.map(tuple)
rdd_fake=dfn.select('description','requirements').filter('fraudulent==1').fillna(' ').rdd.map(tuple)
rdd_real = rdd_real.map(lambda x: x[0]+x[1])
rdd_fake = rdd_fake.map(lambda x: x[0]+x[1])

In [ ]:
rdd=dfn.select('description','requirements').fillna(' ').rdd.map(tuple)

In [ ]:
rdd = rdd.map(lambda x: x[0]+x[1])

In [ ]:
#minor text cleaning
import re
rdd_real = rdd_real.map(lambda x : re.sub('[^a-zA-Z]',' ',x))#keeping just the alphabets
rdd_real = rdd_real.map(lambda x : x.lower())
rdd_fake = rdd_fake.map(lambda x : re.sub('[^a-zA-Z]',' ',x))#keeping just the alphabets
rdd_fake = rdd_fake.map(lambda x : x.lower())

In [ ]:
rdd_fake.first()

In [ ]:
def bigrams_func(line):
    bigrams = []
    unigrams = line.strip().split()
    for i in range(len(unigrams) - 1):
        bigram = unigrams[i] + ":" + unigrams[i+1]
        bigrams.append(bigram)

    return bigrams

In [ ]:
def trigrams_func(line):
    trigrams = []
    unigrams = line.strip().split()
    for i in range(len(unigrams) - 2):
        trigram = unigrams[i] + ":" + unigrams[i+1] + ":" + unigrams[i+2]
        trigrams.append(trigram)

    return trigrams

In [ ]:
from operator import add
#computing bigrams for the real postings
bcount_real = rdd_real.flatMap(bigrams_func).map(lambda x: (x, 1)).reduceByKey(add)
bcount_real.sortBy(lambda x: x[1],ascending=False).take(10)

In [ ]:
#bigrams for fake postings
bcount_fake = rdd_fake.flatMap(bigrams_func).map(lambda x: (x, 1)).reduceByKey(add)
bcount_fake.sortBy(lambda x: x[1],ascending=False).take(10)

In [ ]:
#computing trigrams for real job job postings
tcount_real = rdd_real.flatMap(trigrams_func).map(lambda x: (x, 1)).reduceByKey(add)
tcount_real.sortBy(lambda x: x[1],ascending=False).take(10)

In [ ]:
#computing trigrams for fake job postings
tcount_fake = rdd_fake.flatMap(trigrams_func).map(lambda x: (x, 1)).reduceByKey(add)
tcount_fake.sortBy(lambda x: x[1],ascending=False).take(10)